<a href="https://colab.research.google.com/github/magland/spikeforest_batch_run/blob/master/notebooks/monitor_batches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitor batches

This notebook allow you to monitor the spikeforest processing batches

In [0]:
# Only run this cell if you are running this on a hosted runtime that does not have these packages installed
# %%capture is used to suppress the output... this should take up to a minute to complete
%%capture
!pip install spikeforest

In [0]:
# Import the python packages -- autoreload is used for development purposes
%load_ext autoreload
%autoreload 2

import spikeforest as sf
import spikewidgets as sw
import vdomr as vd
import json

## Configure readonly access to kbucket
sf.kbucketConfigRemote(name='spikeforest1-readonly')

## Load the SpikeForest data
#SF=sf.SFData()
#SF.loadRecordings(key=dict(name='spikeforest_bionet_recordings'))
#SF.loadRecordings(key=dict(name='spikeforest_magland_synth_recordings'))
#SF.loadProcessingBatch(key=dict(batch_name='summarize_recordings_bionet',name='job_results'))
#SF.loadProcessingBatch(key=dict(batch_name='summarize_recordings_magland_synth',name='job_results'))

vdomr: using colab because of VDOMR_MODE environment variable


In [0]:
from kbucket import client as kb
from pairio import client as pa

class BatchJob():
  def __init__(self,object):
    self._object=object
  def command(self):
    return self._object['command']
  def label(self):
    return self._object['label']
  def object(self):
    return self._object
  def getStatus(self):
    val=pa.get(key=self._object)
    if not val:
      return 'not-started'
    if val.startswith('in-process-'):
      return 'in-process'
    elif val.startswith('error-'):
      return 'error'
    result=kb.loadObject(key=self._object)
    if not result:
      return 'result-not-found'
    return 'finished'

class Batch():
  def __init__(self,batch_name):
    self._batch_name=batch_name
    self._batch=kb.loadObject(key=dict(batch_name=batch_name))
  def numJobs(self):
    return len(self._batch['jobs'])
  def job(self,index):
    return BatchJob(self._batch['jobs'][index])

In [0]:
class BatchViewTable(vd.Component):
  def __init__(self):
    vd.Component.__init__(self)
    self._batch=None
  def setBatch(self,batch):
    self._batch=batch
    self.refresh()
  def render(self):
    if not self._batch:
      return vd.span('...')
    
    rows=[]
    header_row=vd.tr(
        vd.th('Job'),
        vd.th('Command'),
        vd.th('Status')
    )
    rows.append(header_row)
    for i in range(self._batch.numJobs()):
      job=self._batch.job(i)
      
      try:
        status=job.getStatus()
      except:
        status='not found'
      
      row=vd.tr(
          vd.td(job.label()),
          vd.td(job.command()),
          vd.td(status)
      )
      rows.append(row)
    table=vd.table(rows,class_='table')
    table2=vd.div(table,style=dict(width='700px',height='400px',overflow='auto'))
    return table2

class BatchView(vd.Component):
  def __init__(self,batch_name):
    vd.devel.loadBootstrap()
    vd.Component.__init__(self)
    self._batch_name=batch_name
    self._batch_view_table=BatchViewTable()
  def _reload_batch(self):
    batch=Batch(batch_name=self._batch_name)
    self._batch_view_table.setBatch(batch)
    self.refresh()
  def reloadBatch(self):
    self._reload_batch()
  def render(self):
    heading=vd.div(
        vd.h3('Batch: '+self._batch_name)
    )
    button_bar=vd.div(
        vd.components.Button('Refresh',onclick=self._reload_batch,class_='btn')
    )
    table=self._batch_view_table
    div=vd.div(
        heading,
        button_bar,
        table
    )
    return div

In [30]:
BV=BatchView(batch_name='ms4_mearec_tetrode')
display(BV)
BV.reloadBatch()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>